In [1374]:
import copy, sys
import logging
logger = logging.getLogger()
logging.getLogger().handlers[:] = []
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(levelname)s: %(message)s',
    force=True
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [1375]:
def show_grid(grid):
    res = ""
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] == 0:
                res += "| "
            elif grid[y][x] < 0:
                res += "|#"
            else:
                res += "|" + str(grid[y][x])
        res += "|\n"
    print(res)

In [1376]:
def show_plans(plans, original_grid):
    for time in range(len(plans[0])):
        grid = copy.deepcopy(original_grid)
        for i, plan in enumerate(plans):
            x, y = plan[time]
            if grid[x][y] < 0:
                logger.error(f"Hit obstacle in plan {i} at time {time}")
            grid[x][y] = i + 1  # Mark the grid with plan index + 1 to differentiate
        show_grid(grid)

# Compute flexibility

In [1377]:
def wait_time_partial_plans(trajectory):
    plan_length = len(trajectory)
    wait_times = [0 for _ in range(plan_length)]
    travel_times = [0 for _ in range(plan_length)]
    subplans = []
    i = 0
    while i < plan_length - 1:
        if trajectory[i] == trajectory[i+1]:
            duration = plan_length - 1
            for j in range(i+1, plan_length):
                if trajectory[j] != trajectory[j-1]:
                    duration = max(j-1, i+1)
                    break
            for k in range(i, duration + 1):
                wait_times[k] = duration - k
            logger.info(f"[Computing wait times] Waiting from time {i} up and till time {j} next time step {duration}")
            i = duration
        else:
            # Gives the number of states that the agent waits
            duration = plan_length - 1
            for j in range(i+1, plan_length):
                if trajectory[j] == trajectory[j-1]:
                    duration = max(j-1, i+1)
                    break
            subplans.append(trajectory[i:j])
            for k in range(i, duration + 1):
                travel_times[k] = duration - k
            logger.info(f"[Computing travel times] Found subplan from time {i} till time {j}: {trajectory[i:j]} next time step {duration}")
            i = duration
    return wait_times, travel_times, subplans

def print_wait_times_and_subplans(trajectories):
    for i, t in enumerate(trajectories):
        res = wait_time_partial_plans(t)
        print(f"Agent {i}: waiting time steps {res[0]} and subplans {res[1]}")

In [1378]:
def flexibility_per_block(trajectories):
    flexibility = [[] for _ in range(len(trajectories))]
    for agent, trajectory in enumerate(trajectories):
        wait_times, _, _ = wait_time_partial_plans(trajectory)
        flexibility[agent] = [0 for _ in range(len(trajectory))]
        for time in range(len(trajectory) - 2, -1, -1):
            m = max(wait_times[time], flexibility[agent][time+1])
            logger.info(f"Agent {agent}, time {time}, trajectory {trajectory} and local recovery time {m}")
            for other_agent, other_trajectory in enumerate(trajectories):
                if agent != other_agent:
                    if trajectory[time] in other_trajectory[time+1:]:
                        index = other_trajectory[time+1:].index(trajectory[time])
                        m = min(m, index)
                        logger.info(f"Recovery time is {m}")
            flexibility[agent][time] = m
    return flexibility

In [1379]:
def print_flexibility(trajectories, print_trajectories=False):
    plan_length = len(trajectories[0])
    if print_trajectories:
        for i, t in enumerate(trajectories):
            print(f"    Agent {i} has initial trajectory {t}")
    flex = flexibility_per_block(trajectories)
    if print_trajectories:
        for i, agent_flex in enumerate(flex):
            for t, f in enumerate(agent_flex):
                if f > 0:
                    # TODO use the actual waiting points
                    new_trajectory = trajectories[i][:t] + [trajectories[i][t]] * f + trajectories[i][t:plan_length - f]
                    print(f"Agent {i} at time {t} can delay up to {f} time steps")
                    for j in range(len(trajectories)):
                        if i == j:
                            print(f"  New trajectory agent {j}: {new_trajectory}")
                        else:
                            print(f"  Old trajectory agent {j}: {trajectories[j]}")
                            for time in range(plan_length):
                                if trajectories[j][time] == new_trajectory[time]:
                                    logger.error(f"Agents {i} and {j} using flexibility of agent {i} both occupy location {trajectories[j][time]} at time {time}")
    return flex

# Block version

# Tests


In [1380]:
logger.setLevel(logging.ERROR)

In [1381]:
def test(flexibility, expected):
    if flexibility != expected:
        logger.error(f"Expected: {expected}")
        logger.error(f"Received: {flexibility}")
        logger.error("ERROR: Flexibility is incorrect")
        raise ValueError("Flexibility is incorrect")
    else:
        print(f"CORRECT flexibility:")
        for i, f in enumerate(flexibility):
            print(f"  Agent {i} has flexibility {f}")

In [1382]:
# Scenario 1 - grid size 4
trajectories = [
    [(0, 0), (1, 0), (2, 0), (3, 0), (3, 0)],
    [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)],
    [(0, 2), (1, 2), (2, 2), (2, 1), (2, 1)]
]
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 1, 1, 0],
]
test(flexibility, expected_flexibility)


CORRECT flexibility:
  Agent 0 has flexibility [0, 0, 0, 1, 0]
  Agent 1 has flexibility [0, 0, 0, 1, 0]
  Agent 2 has flexibility [1, 1, 1, 1, 0]


In [1383]:
# Scenario 2 - corridor
grid = [
    [0, -1, -1, 0],
    [0, 0, 0, 0],
    [0, -1, -1, 0],
]
trajectories = [
    [(0, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 1)],
    [(1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0)]   
]
# show_plans(plans, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [3, 2, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0]
]
test(flexibility, expected_flexibility)
print_wait_times_and_subplans(trajectories)

CORRECT flexibility:
  Agent 0 has flexibility [3, 2, 1, 0, 0, 0]
  Agent 1 has flexibility [0, 0, 0, 0, 1, 0]
Agent 0: waiting time steps [3, 2, 1, 0, 0, 0] and subplans [0, 0, 0, 2, 1, 0]
Agent 1: waiting time steps [0, 0, 0, 0, 1, 0] and subplans [4, 3, 2, 1, 0, 0]


In [ ]:
# Scenario corridor paper
grid = [
    [0, -1, -1, -1, -1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, -1, -1, -1, -1, -1, 0],
]
trajectories = [
    [(0, 0), (0, 0), (0, 0), (1 , 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
    [(2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0)]
]
# show_plans(trajectories, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [2, 1, 0] + [0 for _ in range(7)] + [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
    [9, 8, 7, 6, 5] + [4 for _ in range(13)] + [3, 2, 1, 0]
]
test(flexibility, expected_flexibility)

CORRECT flexibility:
  Agent 0 has flexibility [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
  Agent 1 has flexibility [9, 8, 7, 6, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 0]


In [1385]:
# Scenario three agents same node - grid size 3
trajectories = [
    [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2)],
    [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)],
    [(0, 2), (1, 2), (1, 2), (1, 1), (2, 1)]
]
# show_plans(trajectories, [[0 for _ in range(3)] for _ in range(3)])
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
]
test(flexibility, expected_flexibility)

CORRECT flexibility:
  Agent 0 has flexibility [0, 0, 0, 0, 0]
  Agent 1 has flexibility [0, 0, 0, 1, 0]
  Agent 2 has flexibility [1, 1, 0, 0, 0]


In [1386]:
# Scenario double corridor
grid = [
    [0, -1, -1, -1, 0, -1, -1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, -1, -1, -1, 0, -1, -1, -1, 0],
]
trajectories = [
    [(0, 0), (0, 0), (0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (0, 4), (0, 4), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8)],
    [(2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0)],
    [(2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (1, 8), (1, 7), (1, 6), (1, 5), (1, 4), (2, 4), (2, 4), (2, 4)]
]
show_plans(trajectories, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [2, 1, 0] + [0 for _ in range(5)] + [1] + [0 for _ in range(6)] + [7, 6, 5, 4, 3, 2, 1, 0],
    [7, 6, 5, 4, 3, 2, 1, 1, 1] + [9 for _ in range(5)] + [8, 7, 6, 5, 4, 3, 2, 1, 0],
    [14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3] + [2 for _ in range(8)] + [2, 1, 0]
]
test(flexibility, expected_flexibility)

|1|#|#|#| |#|#|#| |
| | | | | | | | | |
| |#|#|#|2|#|#|#|3|

|1|#|#|#| |#|#|#| |
| | | | | | | | | |
| |#|#|#|2|#|#|#|3|

|1|#|#|#| |#|#|#| |
| | | | | | | | | |
| |#|#|#|2|#|#|#|3|

| |#|#|#| |#|#|#| |
|1| | | | | | | | |
| |#|#|#|2|#|#|#|3|

| |#|#|#| |#|#|#| |
| |1| | | | | | | |
| |#|#|#|2|#|#|#|3|

| |#|#|#| |#|#|#| |
| | |1| | | | | | |
| |#|#|#|2|#|#|#|3|

| |#|#|#| |#|#|#| |
| | | |1| | | | | |
| |#|#|#|2|#|#|#|3|

| |#|#|#| |#|#|#| |
| | | | |1| | | | |
| |#|#|#|2|#|#|#|3|

| |#|#|#|1|#|#|#| |
| | | | |2| | | | |
| |#|#|#| |#|#|#|3|

| |#|#|#|1|#|#|#| |
| | | |2| | | | | |
| |#|#|#| |#|#|#|3|

| |#|#|#| |#|#|#| |
| | |2| |1| | | | |
| |#|#|#| |#|#|#|3|

| |#|#|#| |#|#|#| |
| |2| | | |1| | | |
| |#|#|#| |#|#|#|3|

| |#|#|#| |#|#|#| |
|2| | | | | |1| | |
| |#|#|#| |#|#|#|3|

| |#|#|#| |#|#|#| |
| | | | | | | |1| |
|2|#|#|#| |#|#|#|3|

| |#|#|#| |#|#|#| |
| | | | | | | | |1|
|2|#|#|#| |#|#|#|3|

| |#|#|#| |#|#|#|1|
| | | | | | | | |3|
|2|#|#|#| |#|#|#| |

| |#|#|#| |#|#|#|1|
| | 